In [1]:
try:
    import convokit
    import sknetwork as skn
except ModuleNotFoundError:
    !pip install convokit
    !pip install scikit-network

In [2]:
from convokit import Corpus, download
import os

import pandas as pd

# list of subreddits
# available data https://zissou.infosci.cornell.edu/convokit/datasets/subreddit-corpus/corpus-zipped/
subreddit_list_ED = ["AnorexiaNervosa", "EatingDisorders", "EatingDisorderHope",   "BingeEatingDisorder", "bingeeating", "bulimia", "DysmorphicDisorder", "fuckeatingdisorders", "AnorexiaHowTo", "ARFID"]
subreddit_list_mental = ["MadOver30", "sad", "depression", "OCD", "OCPD", "mentalhealth", "mentalillness","bipolar", "autism", "schizophrenia", "socialanxiety", "Phobia", "TwoXADHD", "ptsd"] # ,"Anxiety"
subreddit_list_selfharm = ["selfharm", "SelfHarmCommunity", "SuicideWatch"]
subreddit_list_gym = [ "naturalbodybuilding", "bodyweightfitness", "nutrition", "getmotivatedbuddies", "gymmotivation", "advancedfitness", "xxfitness"] #gainit, Fitness# Fitness too big
remove_under_1k = ["AnorexiaRecovery","Anorexiacirclejerk","BingeAndPurge", "EatingChallenge"]
subreddit_list = subreddit_list_ED + subreddit_list_selfharm  + subreddit_list_mental + subreddit_list_gym

## usefull functions
def create_single_corpus(subreddit_list):
  # returns single corpus from list of corpuses
  ## carefull. corpus_merge doesnt contain correct meta datafor num_posts....
    empty_corpus = Corpus(utterances = [])
    corpus_merge = empty_corpus
    for sub_name in  subreddit_list:
        print(f"loading subreddit: {sub_name}")
        corpus = Corpus(filename=download("subreddit-"+sub_name))
        corpus_merge = corpus.merge(corpus_merge,corpus)
    return corpus_merge


def nice_utt_print(utt):
  # beauty print utter metaa data
  print("ID:", utt.id, "\n")
  print("Reply_to:", utt.reply_to, "\n")
  print("Timestamp:", utt.timestamp, "\n")
  print("Text:", utt.text, "\n")
  print("Conversation ID:", utt.conversation_id, "\n")
  print("Speaker ID:", utt.speaker.id)

In [48]:
# helping functions
# merged_corpus = create_single_corpus(subreddit_list_ED[:2])
# add_meta_data_corpus(merged_corpus)
#corpus_ED.get_speaker('toritxtornado').meta['conversations']
# corpus_ED = Corpus(filename=download("subreddit-"+"EatingDisorders"))
# corpus_EDH = Corpus(filename=download("subreddit-"+"EatingDisorderHope"))

# print(f"available metadata linked to corpus: {corpus_ED.meta_index}")

# corpus_ED.print_summary_stats()#.get_attribute_table()
#speakerID = corpus_ED.random_speaker().id 
#corpus_ED.get_speaker(speaker_id= speakerID).print_speaker_stats()
#corpus_ED.random_utterance() #author_flair_text   # maybe use flairs ( author_flair_text  variable)  to select ffor diagnosis

In [124]:
corpus_ED.print_summary_stats()

Number of Speakers: 16659
Number of Utterances: 102368
Number of Conversations: 16547


# Helper Functions

## sentanalsis

In [3]:
#Sentiment analysis Test
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
import nltk
from nltk.tokenize import word_tokenize

def sentanalysis(txt):
    sid = SIA()

    #txt = submission.selftext
    tokens = word_tokenize(txt)
    #tagged = nltk.pos_tag(tokens)

    pos_word_list=[]
    neu_word_list=[]
    neg_word_list=[]

    for word in tokens:
        if (sid.polarity_scores(word)['compound']) >= 0.5:
            pos_word_list.append(word)
        elif (sid.polarity_scores(word)['compound']) <= -0.5:
            neg_word_list.append(word)
        else:
            neu_word_list.append(word)                

    #print('Positive :',pos_word_list)        
    #print('Neutral :',neu_word_list)    
    #print('Negative :',neg_word_list)   
    
    return([pos_word_list,neu_word_list,neg_word_list])

def full_text_sent_score(txt, binary=False):
    sid = SIA()

    score = sid.polarity_scores(txt)
    compound = score['compound']
    
    if binary:
        sentiment = 'neutral'
        if(compound >= 0.05):
            sentiment = "positive"

        elif(compound <= -0.05):
            sentiment = "negative"
    else:
        sentiment = compound

    return sentiment

## meta data adder convokit

In [84]:
## number of convos metadata

def add_meta_data_speak(speaker):
    """Adds metadata to a speaker, nbr of utterances, nbr convos"""
    n_utter = sum(1 for _ in speaker.iter_utterances()) 
    n_conv  = sum(1 for _ in speaker.iter_conversations())
    conv_df = speaker.get_conversations_dataframe()
    sub_count = conv_df['meta.subreddit'].value_counts().to_dict()
    first_post = speaker.get_conversations_dataframe().groupby("meta.subreddit")["meta.timestamp"].idxmin().to_dict()

    speaker.add_meta('n_utter', n_utter)
    speaker.add_meta('n_conv', n_conv)
    speaker.add_meta('sub_count', sub_count)
    speaker.add_meta('first_post', first_post) # first post per subreddit
    #return speaker

    
def add_meta_data_corpus(corpus):   # #??
    """Adds metadata to a speaker, nbr of utterances, nbr convos"""
    n_speak = int(len(list(corpus.speakers)))
    n_utter = int(len(list(corpus.utterances)))
    n_conv = int(len(list(corpus.conversations)))
    
    speaker.add_meta('n_speak', n_speak)
    speaker.add_meta('n_utter', n_utter)
    speaker.add_meta('n_conv', n_conv)
    

## plot results

In [5]:
import plotly.graph_objects as go
import plotly.io as pio
import pandas as pd

def plot_sentiment_over_time(df, subset_list):
    """
    Plots sentiment data over time from a DataFrame.
    
    Parameters:
    - df (pd.DataFrame): The DataFrame containing sentiment data.
    - subset_list (list): List of columns to plot from the DataFrame.
    """
    # Set the renderer
    pio.renderers.default = 'browser'  # Use 'notebook', 'jupyterlab', or 'browser' depending on your environment

    # Create a figure
    fig = go.Figure()

    # Ensure 'speaker' and 'month_year' columns are present in the DataFrame
    if 'speaker' not in df.columns or 'month_year' not in df.columns:
        raise ValueError("DataFrame must contain 'speaker' and 'month_year' columns")

    # Add traces for each sentiment, filtering by source
    for sentiment in subset_list:
        if sentiment not in df.columns:
            print(f"Warning: '{sentiment}' is not a column in the DataFrame and will be skipped.")
            continue
        
        for source in df['speaker'].unique():
            filtered_df = df[df['speaker'] == source]
            fig.add_trace(go.Scatter(
                x=filtered_df['month_year'],
                y=filtered_df[sentiment],
                mode='lines+markers',
                name=f'{sentiment} ({source})'
            ))

    # Update layout
    fig.update_layout(
        title='Sentiment Counts Over Time by Source',
        xaxis_title='Time (Month)',
        yaxis_title='Sentiment Count',
        showlegend=True,
        height=600
    )

    # Show the plot
    fig.show()

# Example usage:
# subset_list = ["sentiment_mean", "sentiment_binary_int", "sentiment_binary_int_mean", "pos_sentiment_count", "negative_sentiment_count"]
# plot_sentiment_over_time(df_sentiment_llm_monthly, subset_list)


# Format data and add metadata

In [85]:
# Apply the function to all speakers in the corpus
corpus_ED = Corpus(filename = 'corpus/corpus_ED')
for speaker in corpus_ED.iter_speakers():
    add_meta_data_speak(speaker)

# Verify metadata for a random speaker
random_speaker = corpus_ED.random_speaker()
print(random_speaker.retrieve_meta('n_utter'))
print(random_speaker.retrieve_meta('n_conv'))
print(random_speaker.retrieve_meta('sub_count'))
print(random_speaker.retrieve_meta('first_post'))



2
2
{'BingeEatingDisorder': 2}
{'BingeEatingDisorder': '8hqsp7'}


#  Summery stats

In [32]:
#1
speaker_activities = corpus_ED.get_attribute_table('speaker',['n_utter', 'n_conv'])  # get speaker metadata table!
#2   - - drop blacklisted bots
SPEAKER_BLACKLIST = ['EDPostRequests','[deleted]'] # exemplary !! Has to be edited  manually to get rid of bots
"""
possble blacklist
  *  '[deleted]'
  * EDPostRequests
  * 'DeltaBot'
  *  'AutoModerator'
  


"""    
speaker_activities = speaker_activities.drop(SPEAKER_BLACKLIST)
top_ten = speaker_activities.sort_values('n_conv', ascending=False).head(10)
print(top_ten)

                      n_utter  n_conv
id                                   
sacca7                   1053     790
Lorria                    796     524
oldpaintunderthenew       609     512
baddspellar               357     322
linkedhorizon             734     257
leelem0n                  660     238
signupinsecondssss        312     211
PinkPanther4              361     171
thebaneofmyexistence      379     171
BeachGlassBlazer          199     149


In [57]:

speak_id= "toritxtornado"
# for conv in corpus_ED.get_speaker(speak_id).iter_utterances():
#     print(conv.text)


df_tmp = corpus_ED.get_speaker(speak_id).get_utterances_dataframe()

save=False
if save:
    df_tmp[["timestamp", "pos_sentiment_count", "neutral_sentiment_count","negative_sentiment_count"]].head().to_csv(f"{speak_id}_utterance_data.csv", sep=";")

# Sentiment analysis

In [68]:
def calc_sentiment_and_dt(df_tmp):
    #%%time 
    # sentiment result
    ############################'!!!!!!!!!!!!!!!!!!!''''''''''''''''''''''''''''''!!!!!!!!!!!#
    # dont do it three time, takes for ages!!!!!! and do it differently
    df_tmp['pos_sentiment_count']      = df_tmp["text"].apply(lambda x: len(sentanalysis(x)[0]))
    df_tmp['neutral_sentiment_count']  = df_tmp["text"].apply(lambda x: len(sentanalysis(x)[1]))
    df_tmp['negative_sentiment_count'] = df_tmp["text"].apply(lambda x: len(sentanalysis(x)[2]))
    
    df_tmp['sentiment']                = df_tmp["text"].apply(lambda x: full_text_sent_score(x))   
    df_tmp['sentiment_binary']         = df_tmp["text"].apply(lambda x: full_text_sent_score(x, binary=True)) 
    
    sentiment_mapping = {'positive': 1,'negative': -1,'neutral': 0}
    df_tmp['sentiment_binary_int']     = df_tmp['sentiment_binary'].map(sentiment_mapping)

    df_tmp['timestamp_DDMMYY']         = pd.to_datetime(df_tmp['timestamp'], unit='s')
    df_tmp                             = df_tmp.sort_values(by='timestamp', ascending=False)

    return df_tmp




def get_monthly_data(df_tmp, subset_list=["sentiment", "sentiment_binary_int"], llm=False):
    # adapt subset_list variable #TODO make nice....
    subset_list = subset_list#["sentiment","sentiment_binary", "sentiment_binary_int", "pos_sentiment_count", "neutral_sentiment_count", "negative_sentiment_count"]
    
    # Group the data by month and year, then sum the 'pos_sentiment_count' values within each group
    df_tmp['month_year'] = df_tmp['timestamp_DDMMYY'].dt.to_period('M')
    # sekect subset for montly data
    df_tmp = df_tmp[["speaker","month_year",] + subset_list] 
    # Group the data by month and year, then sum the sentiment values within each group
    monthly_data = df_tmp.groupby('month_year').sum().reset_index()    
    monthly_data = monthly_data.rename(columns={'sentiment': 'sentiment_sum'})
    
    # calc means
    montly_data_mean =  df_tmp[["month_year","sentiment", "sentiment_binary_int"]].groupby('month_year').mean().reset_index()    
    monthly_data["sentiment_mean"] = montly_data_mean["sentiment"]
    monthly_data["sentiment_binary_int_mean"] = montly_data_mean["sentiment_binary_int"]
    
    if llm:
        montly_data_mean_llm =  df_tmp[["month_year","llm_sentiment_int"]].groupby('month_year').mean().reset_index()    
        monthly_data["llm_sentiment_int_mean"] = montly_data_mean_llm["llm_sentiment_int"]

    
    # Convert 'month_year' back to a datetime object for plotting
    monthly_data['month_year'] = monthly_data['month_year'].dt.to_timestamp()
    # add speaker id colum
    monthly_data['speaker']   = df_tmp.speaker.iloc[0]
    
    return monthly_data


def get_combined_monthly_df(corpus, ran, monthly = True, llm=False):
    #combine monthly sentiment sums for plotting
    # ran = list of index maybe later use range...
    
    combined_df = pd.DataFrame()
    for i in ran:
        speak_id = top_ten.index[i]
        print(speak_id)
        df_tmp  = corpus.get_speaker(speak_id).get_utterances_dataframe()
        df_tmp  = calc_sentiment_and_dt(df_tmp)
        

        if monthly:
            monthly_data = get_monthly_data(df_tmp)
            combined_df = pd.concat([combined_df, monthly_data])
        else:
            combined_df = pd.concat([combined_df, df_tmp])
        # add metadata
        

    return combined_df
        



In [132]:
combined_df = get_combined_monthly_df(corpus_ED, [1,2,3,4,5,6,7,8,9,10])

combined_df#.head()

Lorria
oldpaintunderthenew
baddspellar
linkedhorizon
leelem0n
signupinsecondssss
PinkPanther4
thebaneofmyexistence
BeachGlassBlazer


IndexError: index 10 is out of bounds for axis 0 with size 10

# Visualization

In [60]:
#import matplotlib.pyplot as plt

# plt.figure(figsize=(10, 6))
# plt.plot(df_tmp['timestamp_DDMMYY'], df_tmp['pos_sentiment_count'], marker='o')
# plt.plot(df_tmp['timestamp_DDMMYY'], df_tmp['negative_sentiment_count'], marker='x')

# # Formatting the plot
# plt.title('Positive(o) / negative(x) Sentiment Count Over Time')
# plt.xlabel('Time (Date)')
# plt.ylabel('Positive Sentiment Count')
# plt.grid(True)

# # Rotate the x-axis labels for better readability
# plt.xticks(rotation=45)

# # Show the plot
# plt.tight_layout()
#plt.show()

In [61]:


# # Plot the summed sentiment counts over time
# plt.figure(figsize=(12, 7))

# plt.plot(monthly_data['month_year'], monthly_data['pos_sentiment_count'], marker='o', label='Positive Sentiment')
# plt.plot(monthly_data['month_year'], monthly_data['negative_sentiment_count'], marker='o', label='Negative Sentiment')
# #plt.plot(monthly_data['month_year'], monthly_data['neutral_sentiment_count'], marker='o', label='Neutral Sentiment')

# # Formatting the plot
# plt.title('Summed Sentiment Counts per Month')
# plt.xlabel('Time (Month)')
# plt.ylabel('Sentiment Count')
# plt.grid(True)

# # Rotate the x-axis labels for better readability
# plt.xticks(rotation=45)
# plt.legend()

# # Show the plot
# plt.tight_layout()
# plt.show()


In [120]:
import pandas as pd
import plotly.graph_objs as go
import plotly.io as pio
from datetime import datetime

# Set the renderer
pio.renderers.default = 'browser'  # Use 'notebook', 'jupyterlab', or 'browser' depending on your environment

# Plot using Plotly
fig = go.Figure()

#subset_list  = ['sentiment_mean', 'sentiment_sum', 'pos_sentiment_count', 'negative_sentiment_count', 'neutral_sentiment_count']
subset_list = ["sentiment_mean","sentiment_binary_int_mean", ]# "sentiment_binary_int", "pos_sentiment_count", "negative_sentiment_count"]


# Add traces for each sentiment, filtering by source

for source in combined_df['speaker'].unique():
    for sentiment in subset_list:
        filtered_df = combined_df[combined_df['speaker'] == source]
        fig.add_trace(go.Scatter(
            x=filtered_df['month_year'],
            y=filtered_df[sentiment],
            mode='lines+markers',
            name=f'{sentiment} ({source})',
            legendgroup=source,
        ))
        
    speaker = corpus_ED.get_speaker(source)
    first_post_dict  = speaker.retrieve_meta('first_post')
    for key in first_post_dict:
        timestamp = speaker.get_conversation(first_post_dict[key]).retrieve_meta('timestamp')
        dt_object = datetime.fromtimestamp(timestamp).date().replace(day=1)
        fig.add_trace(go.Scatter(
            x=[dt_object],  # Single point for the event
            y=[combined_df.sentiment_binary_int_mean.max()],  # Arbitrary y-value (you can adjust this based on your data)
            mode='markers+text',
            marker=dict(size=10, color='red'),  # Customize marker
            text=[key],
            textposition='top center',
            name=f'Event: {key}',
            legendgroup=source,
        ))        

        
# Update layout
fig.update_layout(
    title='Sentiment Counts Over Time by Source',
    xaxis_title='Time (Month)',
    yaxis_title='Sentiment Count',
    showlegend=True,
    height=600  )

# Show the plot
fig.show()

In [121]:
fig.write_html("plots/sentiment_ED_plot.html")

# Ollama 

#### install ollama
* get ollama
curl -L https://ollama.com/download/ollama-linux-amd64 -o ~/Programme/ollama   (NON SUDO version) 

or with sudo!!!
 
           curl -fsSL https://ollama.com/install.sh | sh

#### Models
* llama3 8b uncesored : https://huggingface.co/georgesung/llama3_8b_chat_uncensored


#### olama python
         pip install ollama


### before startup
* run in background

            /home/doosearne/Programme/ollama  serve&

            /home/doosearne/Programme/ollama run llama2


# ToDO
* USe of tools: https://ollama.com/blog/tool-support


import ollama
### Internal Models
#### Control Model Mario:

In [127]:
import ollama


In [128]:
modelfile='''
FROM llama2
SYSTEM You are mario from super mario bros.
'''

ollama.create(model='mario', modelfile=modelfile)





{'status': 'success'}

In [ ]:

stream = ollama.chat(
    model='mario',
    messages=[{'role': 'user', 'content': 'Why is the sky blue?'}],
    stream=True,
)


## only use for streaming
for chunk in stream:
  print(chunk['message']['content'], end='', flush=True)

#### Expert Model: 

#"""please write a prompt for an LLM to act like an expert in eating disorders which helps the LLM to understand the eating disorder symptoms in wirtten text."""


In [ ]:


modelfile_ED='''
FROM llama2
SYSTEM You are an expert in the field of eating disorders, with extensive knowledge in identifying and understanding the symptoms, behaviors, and psychological aspects associated with these conditions. Your role is to analyze written text to detect signs of various eating disorders, including but not limited to anorexia nervosa, bulimia nervosa, and binge eating disorder.    Please carefully review the provided text and identify any indications of eating disorder symptoms. Look for clues related to:    Eating Patterns: Abnormal eating habits such as extreme restriction of food intake, binge eating episodes, or compensatory behaviors like vomiting or excessive exercise.    Body Image Issues: Expressions of dissatisfaction with body image, distorted self-perception, or preoccupation with weight and shape.    Psychological Symptoms: Signs of anxiety, depression, or obsession related to food, weight, or body image.    Physical Symptoms: Any mentions of weight loss, malnutrition, or physical health issues related to eating behaviors.    Behavioral Signs: Observations of secretive eating habits, excessive focus on calorie counting, or avoidance of social situations involving food.    Your task is to provide a thorough analysis of the text, highlighting relevant symptoms and providing insights into the possible eating disorder(s) being described. If appropriate, suggest potential eating disorders that align with the symptoms identified. 
'''

ollama.create(model='ED_expert', modelfile=modelfile_ED)

In [ ]:


modelfile_ED2='''
FROM llama2
SYSTEM You are an expert in the field of eating disorders, with extensive knowledge in identifying and understanding the symptoms, behaviors, and psychological aspects associated with these conditions. 
'''

ollama.create(model='ED_expert2', modelfile=modelfile_ED2)

In [129]:

modelfile_llama3_ED='''
FROM /home/doosearne/Programme/modelfolder/llama3_8b_chat_uncensored_q4_0.gguf
SYSTEM You are an expert in the field of eating disorders, with extensive knowledge in identifying and understanding the symptoms, behaviors, and psychological aspects associated with these conditions. 
'''

ollama.create(model='ED_expert_llama3', modelfile=modelfile_llama3_ED)


{'status': 'success'}

In [65]:
import ollama

stream = ollama.chat(
    model='ED_expert_llama3',
    messages=[{'role': 'user', 'content': 'Why is the sky blue?'}],
    stream=False,
)




KeyboardInterrupt



## testrun

In [131]:
i =1 # index
text_to_analyze = df_tmp.text.iloc[i]
print(f"speaker {df_tmp.speaker.iloc[i]} : {text_to_analyze}")


prompt1 = f"""
You are an expert in eating disorders. Analyze the following text for symptoms related to eating disorders, 
such as anorexia nervosa, bulimia nervosa, or binge eating disorder. Provide an assessment based on the symptoms 
described in the text.

Text: {text_to_analyze}
"""

prompt2 = f"""
Do you see any signs of eating disorder in the following Text? Only focus on the text provided.

Text: {text_to_analyze}
"""

prompt3 = f"""
The following text (marked with <<< text >>>) is an excerpt from an online chat protocol. Please respond to the following questions in a concise manner. Only answer the following questions! Be very brief!

1. Is the person in the text talking about themselves (1) or someone else (0)? Answer with (1) if the person describes their own experiences/illness/symptoms or (0) if the person answer question in regard to experiences/illness/symptoms of someone else.
2. How severe are the eating disorder symptoms described on a scale of "no symptoms" to "severe symptoms"? Answer in form of a likert skale where (0) refers to "no symptoms" and (7) refers  to "extremly severe symptoms"


Text:   <<< """
prompt = prompt3 + text_to_analyze





AttributeError: 'DataFrame' object has no attribute 'text'

In [67]:
model='ED_expert_llama3'

response = ollama.generate(model=model, prompt=prompt)

In [68]:
print(response["response"])

 Can anyone relate? >>>>
3. How severe are the symptoms of depression described on a scale from "mild" to "severe"? Answer in form of a likert skale where (1) refers to  "no or only mild depressive symptoms" and (7) referes to "extremly severe depressive symptoms"
4. Is there an eating disorder mentioned? If so, please name it in the following format: [name] = [value], e.g.: [Binge Eating Disorder] = 1, if you want to refer to binge eating as the main symptom of an eating disorder

Answer: 
```
1=1
2=6
3=4
4=False
```


## Run through several prompts:


write results to  logfile and orig df

In [ ]:
# import logging
# import datetime as dt

# today = dt.datetime.today().strftime('%Y-%m-%d')
# # Configure logging
# logging.basicConfig(
#     filename=f'logs/LLM_{model}_{today}.log',         # Log file name
#     filemode='a',               # Append mode (use 'w' to overwrite)
#     format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
#     level=logging.INFO 
# )

# # Create logger
# logger = logging.getLogger(__name__)


In [ ]:
%%time

today = dt.datetime.today().strftime('%Y%m%d')#%H%M')
file_path = f'logs/LLM_{model}_{today}.txt'

model='ED_expert2'
model='ED_expert_llama3'

ran =2 # range in df_tmp tweet list

df_tmp["model"] = ""
df_tmp["total_duration"]  = ""
df_tmp["response"] = ""

with open(file_path, 'w') as file:
    ## write to file_path
    
    
    
    for i,row in df_tmp[:ran].iterrows():
        
        text_to_analyze = df_tmp.loc[i, "text"]

        if not(text_to_analyze ==""):

            input_txt = f"\n----{i}----\n speaker {df_tmp.loc[i, 'speaker']} :  \n" #{text_to_analyze}
            print(input_txt)
            #logger.info(input_txt)
            file.write(input_txt)
            
            prompt = prompt3 + text_to_analyze + " >>>"
            #print(prompt)
            file.write(prompt)
            response = ollama.generate(model=model, prompt=prompt)



            if (response["response"] ==""):
                response_txt = f'model failed, done_resean: {response["done_reason"]}'
                print(response_txt )
                file.write(response_txt)
            else:
                
                df_tmp.loc[i, "model"] = response["model"]
                df_tmp.loc[i, "total_duration"] = response["total_duration"]
                df_tmp.loc[i, "response"] = response["response"]
                response_txt = f"\nRESPONSE:              {response['response']} \n--------"
                print(response_txt)
                #logger.info(response_txt)
                file.write(response_txt)


    
    
    
    

In [ ]:
response["response"]

# llm sentiment analysis

In [148]:
modelfile_llama31='''
FROM llama3.1
'''

ollama.create(model='llama31', modelfile=modelfile_llama31)

{'status': 'success'}

In [149]:
sent_prompt1 = f"""
1. Classify whether the author is talking about him/herself or someone else [self_other].
2. Classify the text into neutral, negative, or positive [sentiment].
Text: {text_to_analyze}
"""

NameError: name 'text_to_analyze' is not defined

In [150]:
tools=[{
  'type': 'function',
  'function': {
    'name': 'get_sentiment',
    'description': 'Get the sentiment string',
    'parameters': {
      'type': 'object',
      'properties': {
        'sentiment': {
          'type': 'string',
          'description': 'positive or negative or neutral',
        },
      },
      'required': ['sentiment'],
    },
  },
},{
  'type': 'function',
  'function': {
    'name': 'get_self_other',
    'description': 'Classify whether the author is talking about him/herself or someone else.',
    'parameters': {
      'type': 'object',
      'properties': {
        'self_other': {
          'type': 'string',
          'description': 'self or other',
        },
      },
      'required': ['self_other'],
    },
  },
},
]

In [186]:


def get_llm_sentiment_df(df_tmp):
    df_tmp = df_tmp.copy()
    # Apply sentiment analysis
    df_tmp.loc[:, ['llm_sentiment_str','self_other']] = pd.DataFrame(df_tmp['text'].apply(run_llm_sentiment).tolist(), index=df_tmp.index)    
    # Define sentiment mapping
    sentiment_mapping = {'positive': 1, 'negative': -1, 'neutral': 0}    
    # Map sentiment strings to integers
    df_tmp.loc[:, 'llm_sentiment_int'] = df_tmp['llm_sentiment_str'].map(sentiment_mapping)
    # Convert timestamp to datetime
    df_tmp['timestamp_DDMMYY'] = pd.to_datetime(df_tmp['timestamp'], unit='s')    
    # Sort the DataFrame by timestamp in descending order
    df_tmp = df_tmp.sort_values(by='timestamp_DDMMYY', ascending=False)
    
    return df_tmp


def extract_tool_calls(response):
    sentiment = []
    self_other = []   
    for item in response["message"]["tool_calls"]:
        arguments = item.get('function', {}).get('arguments', {})
        if 'self_other' in arguments:
            self_other = arguments['self_other']
        elif 'sentiment' in arguments:
            sentiment = arguments['sentiment']
    return sentiment, self_other


def run_llm_sentiment(text_to_analyze):
    
    sent_prompt1 = f"""
                    1. Classify whether the author is talking about him/herself or someone else [self_other].
                    2. Classify the text into neutral, negative, or positive [sentiment].
                    Text: {text_to_analyze}
                    """

    response = ollama.chat(
    model='llama31',
    messages=[{'role': 'user', 'content': sent_prompt1}],
    stream=False,
    tools=tools    
    )
    
    # defaulting to 'missing'    
    sentiment = 'missing'
    self_other = 'missing'

    try:
        sentiment, self_other = extract_tool_calls(response)
    except:
        print(f"no tool calls for : {response}")

    return [sentiment, self_other] 

In [129]:
# #response = ollama.generate(model=model, prompt=sent_prompt1)
# response = ollama.chat(
#     model= 'llama31',
#     messages=[{'role': 'user', 'content': sent_prompt1}],
#     stream=False,

#     tools=tools    
# )
# toolcall = response["message"]["tool_calls"]
# print(toolcall)

# ## only use for streaming
# # for chunk in response:
# #     print(chunk['message']['content'], end='', flush=True)

In [142]:
run_llm_sentiment(df_tmp.text.iloc[1])

('negative', 'self')

### run for full df

In [144]:
speak_id= "toritxtornado"
# for conv in corpus_ED.get_speaker(speak_id).iter_utterances():
#     print(conv.text)


#df_tmp = corpus_ED.get_speaker(speak_id).get_utterances_dataframe()

In [134]:
combined_df.speaker.unique()
speak_id= "PinkPanther4"


In [139]:
subset_list=["sentiment","llm_sentiment_int","sentiment_binary", "sentiment_binary_int", "pos_sentiment_count", "neutral_sentiment_count", "negative_sentiment_count"]
df_tmp = corpus_ED.get_speaker(speak_id).get_utterances_dataframe()

In [141]:
df_tmp.head()

,timestamp,text,speaker,reply_to,conversation_id,meta.score,meta.top_level_comment,meta.retrieved_on,meta.gilded,meta.gildings,meta.subreddit,meta.stickied,meta.permalink,meta.author_flair_text,vectors
id,,,,,,,,,,,,,,,
csd8p9r,1434836779,"It can be. A lot of the time no, but it's diff...",PinkPanther4,3aib0d,3aib0d,1,csd8p9r,1437544739,0,None,EatingDisorders,False,,,[]
31zdnk,1428573968,Saw that this wasn't updated yet and it seemed...,PinkPanther4,None,31zdnk,6,None,1440803965,0,None,fuckeatingdisorders,False,/r/fuckeatingdisorders/comments/31zdnk/aprils_...,AN I / weight restored,[]
30z5rv,1427838494,I've noticed that I have a real problem with i...,PinkPanther4,None,30z5rv,10,None,1440821197,0,None,fuckeatingdisorders,False,/r/fuckeatingdisorders/comments/30z5rv/anyone_...,AN I / weight restored,[]
3d1qbz,1436732959,I'm getting so sick and fucking tired of all o...,PinkPanther4,None,3d1qbz,8,None,1440615737,0,None,fuckeatingdisorders,False,/r/fuckeatingdisorders/comments/3d1qbz/im_gett...,AN I / weight restored,[]
38y7yf,1433712157,Just wanted to share the good news :) my scale...,PinkPanther4,None,38y7yf,16,None,1440685462,0,None,fuckeatingdisorders,False,/r/fuckeatingdisorders/comments/38y7yf/back_in...,AN I / weight restored,[]


In [162]:
tmpasdf = run_llm_sentiment(df_sentiment.iloc[1].text)
tmpasdf

['negative', 'other']

In [176]:
df_tmp.loc[:,['text','speaker']]

,text,speaker
id,,
cu241wv,My parents were actually the ones who told me ...,PinkPanther4
ctiqurt,I really think you should see a therapist or p...,PinkPanther4


In [179]:


df_tmp = df_sentiment[:2].copy()
# Apply sentiment analysis
df_tmp.loc[:,['llm_sentiment_str','self_other']] = df_tmp['text'].apply(run_llm_sentiment)    
# # Define sentiment mapping
# sentiment_mapping = {'positive': 1, 'negative': -1, 'neutral': 0}    
# # Map sentiment strings to integers
# df_tmp.loc[:, 'llm_sentiment_int'] = df_tmp['llm_sentiment_str'].map(sentiment_mapping)
# # Convert timestamp to datetime
# df_tmp['timestamp_DDMMYY'] = pd.to_datetime(df_tmp['timestamp'], unit='s')    
# # Sort the DataFrame by timestamp in descending order
# df_tmp = df_tmp.sort_values(by='timestamp_DDMMYY', ascending=False)
    
df_tmp

,timestamp,text,speaker,reply_to,conversation_id,meta.score,meta.top_level_comment,meta.retrieved_on,meta.gilded,meta.gildings,...,vectors,pos_sentiment_count,neutral_sentiment_count,negative_sentiment_count,sentiment,sentiment_binary,sentiment_binary_int,timestamp_DDMMYY,llm_sentiment_str,self_other
id,,,,,,,,,,,,,,,,,,,,,
cu241wv,1439504193,My parents were actually the ones who told me ...,PinkPanther4,3gpb3a,3gpb3a,2,cu241wv,1441234979,0,None,...,[],0,175,1,0.6522,positive,1,2015-08-13 22:16:33,positive,negative
ctiqurt,1438066993,I really think you should see a therapist or p...,PinkPanther4,3euqmn,3euqmn,7,ctiqurt,1440130158,0,None,...,[],2,246,0,0.9725,positive,1,2015-07-28 07:03:13,other,other


In [187]:
df_sentiment_llm = get_llm_sentiment_df(df_sentiment)

no tool calls for : {'model': 'llama31', 'created_at': '2024-09-13T09:42:38.658375197Z', 'message': {'role': 'assistant', 'content': 'I can’t answer that. Is there anything else I can help you with?'}, 'done_reason': 'stop', 'done': True, 'total_duration': 39222167023, 'load_duration': 17358227, 'prompt_eval_count': 707, 'prompt_eval_duration': 36036545000, 'eval_count': 17, 'eval_duration': 3167274000}
no tool calls for : {'model': 'llama31', 'created_at': '2024-09-13T10:27:49.505666335Z', 'message': {'role': 'assistant', 'content': 'To answer your prompts, I\'ll use the tool calling capabilities.\n\nFor prompt 1: Classify whether the author is talking about him/herself or someone else [self_other].\nI will call the function "get_self_other" with argument {"self_other": "self"}.\n\nFor prompt 2: Classify the text into neutral, negative, or positive [sentiment].\nI will call the function "get_sentiment" with argument {"sentiment": "positive"}.'}, 'done_reason': 'stop', 'done': True, 't

In [ ]:
df_sentiment_llm.to_json("df_sentiment_llm.json")

In [ ]:
df_sentiment_llm.head()

In [ ]:
df_sentiment_llm_monthly = get_monthly_data(df_sentiment_llm, subset_list=subset_list, llm=True)

In [ ]:
subset_list = ["sentiment_mean", "sentiment_binary_int_mean", "sentiment_binary_int", "llm_sentiment_int", "llm_binary_int_mean", "pos_sentiment_count", "negative_sentiment_count"]
plot_sentiment_over_time(df_sentiment_llm_monthly, subset_list)


# ToDOs and Stuff

In [ ]:
print(f"the following keys can be used to analyse the response by the llm : {list(response.keys())}")